In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [2]:
path = os.getcwd() + "/atribuna/milarquivos"
content_file_array = []
classifier_array = []

In [3]:
def read_text(path, file_Name):
    with open(path, encoding="latin-1") as file:
        content_file = file.read()
        Build_DataFrame(content_file, file_Name)

In [4]:
def Build_DataFrame(content_file, file_Name):
    classifier =  GetClassifier(file_Name) 
    if(len(classifier) >= 3):        
        classifier_array.append(classifier)
        content_file_array.append(content_file)  

In [5]:
def GetClassifier(file_Name):
    name_split =  file_Name.split(".")
    first_name = name_split[0];
    return first_name[(len(first_name) - 3) : len(first_name)]

In [6]:
for file_Name in tqdm(os.listdir(path), desc="Carregar arquivos"):    
    if file_Name.endswith(".txt"):
        file_path = f"{path}/{file_Name}"
      
        # call read text file function
        read_text(file_path, file_Name)

Carregar arquivos: 100%|███████████████████████████████████████████████████████████| 154/154 [00:00<00:00, 8337.93it/s]


In [7]:
textDataDrame =  pd.DataFrame({"Text":content_file_array, "Class":classifier_array})


In [8]:
textDataDrame.head(10)

,Text,Class
0,Nasce um novo amiguinho virtual\n<b> Coelhinho...,inf
1,Brigas na saída de boates\n<b> A denúncia é de...,poc
2,Queijo fica perto do livro dos recordes\n<b> O...,poc
3,Queijo fica perto do livro dos recordes\n<b> O...,reg
4,PLENÁRIO\n<b> </b><br>\n <br>\nBolo social do...,pot
5,Quadrilha assalta lanchonete\n<b> Cinco bandid...,poc
6,CARTAS\n<b> </b><br>\n <br>\nParabéns\nÀ Rede...,opi
7,Mineiro é baleado em assalto \n<b> O gerente c...,poc
8,CARTAS\n<b> </b><br>\n <br>\nParabéns\nVenho ...,opi
9,Aluno é morto com um soco\n<b> Adolescente de ...,poc


# StopWords removing

In [9]:
#!conda install unidecode

import nltk
import unidecode
from bs4 import BeautifulSoup

# nltk.download()
#- averaged_perceptron_tagger
# - floresta
# - mac_morpho
# - machado
# - punkt
# - stopwords
# - wordnet
# - words

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [10]:
def addStopWordsToList(texto):
    example_sent = texto
    word_tokens = word_tokenize(example_sent)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = ""

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence += w + " "
    return filtered_sentence.strip()

In [11]:
stop_words = set(stopwords.words('portuguese'))
def returnDataFrameStopWords(dataFrame):
    dataReturn = pd.DataFrame()
    for row in tqdm(dataFrame.itertuples(index=True, name='Pandas'), desc="Limpeza de texto"):
        Text = row.Text
        Class = row.Class

        Text = BeautifulSoup(Text, "lxml").text
        Text = unidecode.unidecode(Text)
        Text = addStopWordsToList(Text)
        Text = Text.replace(" . "," ")
        Text = Text.replace(" , "," ")
        Text = Text.replace(" - "," ")
        Text = Text.replace(" : "," ")
        Text = Text.lower()

        linha = pd.Series([Text, Class])
        linha = pd.DataFrame([linha])
        dataReturn = pd.concat([linha, dataReturn], ignore_index=True)

    return dataReturn

In [12]:
df_text_clean = returnDataFrameStopWords(textDataDrame)

Limpeza de texto: 152it [00:00, 189.73it/s]


## Fatores de Ponderação

In [13]:
df_text_clean = df_text_clean.rename(columns = {0: 'Text', 1: 'Class'}, inplace = False)
df_text_clean.head()

,Text,Class
0,acusado trafico morto maria ortiz o acusado tr...,poc
1,notas capixaba maxwell inter milao o inter mil...,ept
2,portugal rende familia scolari os jogadores im...,ept
3,briga pegar pescoco fica dito nao dito a tribu...,ept
4,fama nao garante vaga melbourne o elenco austr...,ept


In [14]:
from sklearn.feature_extraction.text import TfidfTransformer # tf-idf
from sklearn.feature_extraction.text import CountVectorizer # tf

In [15]:
classes = []
for row in df_text_clean.itertuples(index=True, name='Pandas'):  
    if row.Class not in classes:
        classes.append(row.Class)
classes

['poc',
 'ept',
 'pot',
 'cit',
 'int',
 'eco',
 'opi',
 'inf',
 'reg',
 'bro',
 'cid',
 'at2',
 'sro']

In [16]:
 classes =  ["at2","sro","inf","imo","ept","esp","fam","tvt","cid","mul","pot","tav","mic","opi","poc","reg","eco","bro","cit","con","int"]

## TF

In [17]:
vectorizer = CountVectorizer()
TF = vectorizer.fit_transform(df_text_clean.Text)
features_names =  vectorizer.get_feature_names()

## TF-IDF

In [18]:
from sklearn.pipeline import Pipeline

In [19]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=features_names)),
                 ('tfid', TfidfTransformer())]).fit(df_text_clean.Text)

# Lista de palavras

In [20]:
lista_palavras = []
for row in tqdm(df_text_clean.itertuples(index=True, name='Pandas')):
    palavras = row.Text.split(" ")
    for palavra in palavras:
        if(not len(palavra) <= 2):
            if(palavra not in lista_palavras):
                lista_palavras.append(palavra)

152it [00:01, 130.99it/s]


---

# frequencia palavra por classe

In [21]:
dic_doc_class = {}

for classe in tqdm(classes):
    lista_doc = []
    for row in df_text_clean.itertuples(index=True, name='Pandas'):  
        if (str(classe) == row.Class):
            lista_doc.append(row.Text)
    dic_doc_class[f'{classe}'] = lista_doc

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1364.83it/s]


In [22]:
df_relacao_class = pd.DataFrame()
df_relacao_class['Index'] = lista_palavras
for key in tqdm(dic_doc_class):
    lista_docs =  dic_doc_class[key]
    docs = ""
    for doc in lista_docs:
        docs += f' {doc}'
        
    doc = docs
    
    lista_quant =[]
    
    for termo in lista_palavras:
        quant = doc.split(" ").count(termo)
        lista_quant.append(quant)
        
    df_relacao_class[f'{key}'] = lista_quant
    
    
df_relacao_class = df_relacao_class.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:17<00:00,  1.17it/s]


In [23]:
df_relacao_class.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,0,0,0,0,0,0,0,0,0,0,...,0,0,0,23,0,0,0,0,0,0
trafico,0,0,0,0,0,0,0,0,0,0,...,0,0,1,7,0,0,0,0,0,0
morto,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12,0,0,0,0,0,0
maria,2,0,0,0,1,0,0,0,0,0,...,0,0,8,6,2,2,0,0,0,0
ortiz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


# ni - o numero  de documentos em que um determinado termo ki aparece

In [24]:
dic_ni = {}

for termo in tqdm(lista_palavras):
    quant = 0 
    for row in df_text_clean.itertuples(index=True, name='Pandas'):
        doc = row.Text
        if(termo in doc):
            quant += 1
    dic_ni[f'{termo}'] = quant

100%|████████████████████████████████████████████████████████████████████████████| 9561/9561 [00:08<00:00, 1103.64it/s]


# a - quantidade de documentos que pertence a uma classe e posuem o termo ki

#  c - o numero  de documentos que pertencem a Cp, mas não possuem o termo ki 

In [25]:
df_a = pd.DataFrame()
df_a['Index'] = lista_palavras

df_c = pd.DataFrame()
df_c['Index'] = lista_palavras

for key in tqdm(dic_doc_class):
    lista_docs =  dic_doc_class[key]
    lista_quant_a =[]
    lista_quant_c =[]

    for termo in lista_palavras:
        quant_a = 0
        quant_c = 0
        
        for doc in lista_docs:
            if(termo in doc):
                quant_a += 1
            else:
                quant_c += 1
                
        lista_quant_a.append(quant_a)
        lista_quant_c.append(quant_c)
        
    df_a[f'{key}'] = lista_quant_a
    df_c[f'{key}'] = lista_quant_c
    
    
df_a = df_a.set_index('Index')
df_c = df_c.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:02<00:00,  8.76it/s]


# df_a

In [26]:
df_a.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,0,0,0,0,0,0,0,0,0,0,...,0,0,0,14,0,0,0,0,0,1
trafico,0,0,0,0,0,0,0,0,0,0,...,0,0,1,5,0,0,0,0,0,0
morto,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,1
maria,1,0,0,0,1,0,0,0,1,0,...,0,0,5,2,1,1,0,0,0,0
ortiz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# df_c

In [27]:
df_c.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,9,1,4,0,27,0,0,0,7,0,...,0,0,7,23,7,13,4,6,0,15
trafico,9,1,4,0,27,0,0,0,7,0,...,0,0,6,32,7,13,4,6,0,16
morto,9,1,4,0,27,0,0,0,7,0,...,0,0,7,32,7,13,4,6,0,15
maria,8,1,4,0,26,0,0,0,6,0,...,0,0,2,35,6,12,4,6,0,16
ortiz,9,1,4,0,27,0,0,0,7,0,...,0,0,7,36,7,13,4,6,0,16


# b - a quantidade de documentos que possuem o termo ki, mas não pertence a Cp

# d o numero de documentos que não possuem ki e não pertencem a Cp.


In [28]:
df_b = pd.DataFrame()
df_b['Index'] = lista_palavras

df_d = pd.DataFrame()
df_d['Index'] = lista_palavras

for key in tqdm(dic_doc_class):
    lista_quant_b =[]
    lista_quant_d =[]
    
    for termo in lista_palavras:
        quant_b = 0
        quant_d = 0
        for classe in classes:
            if(classe != key):
                lista_docs =  dic_doc_class[classe]
                
                for doc in lista_docs:
                    if(termo in doc):
                        quant_b += 1
                    else:
                        quant_d += 1
                        
        lista_quant_b.append(quant_b)
        lista_quant_d.append(quant_d)
        
    df_b[f'{key}'] = lista_quant_b
    df_d[f'{key}'] = lista_quant_d
    
df_b = df_b.set_index('Index')
df_d = df_d.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:45<00:00,  2.19s/it]


# df_b

In [29]:
df_b.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,17,17,17,17,17,17,17,17,17,17,...,17,17,17,3,17,17,17,17,17,16
trafico,7,7,7,7,7,7,7,7,7,7,...,7,7,6,2,7,7,7,7,7,7
morto,6,6,6,6,6,6,6,6,6,6,...,6,6,6,1,6,6,6,6,6,5
maria,12,13,13,13,12,13,13,13,12,13,...,13,13,8,11,12,12,13,13,13,13
ortiz,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1


# df_d

In [30]:
df_d.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,126,134,131,135,108,135,135,135,128,135,...,135,135,128,112,128,122,131,129,135,120
trafico,136,144,141,145,118,145,145,145,138,145,...,145,145,139,113,138,132,141,139,145,129
morto,137,145,142,146,119,146,146,146,139,146,...,146,146,139,114,139,133,142,140,146,131
maria,131,138,135,139,113,139,139,139,133,139,...,139,139,137,104,133,127,135,133,139,123
ortiz,142,150,147,151,124,151,151,151,144,151,...,151,151,144,115,144,138,147,145,151,135


---

In [31]:
def get_row_class(row, classe):
    dic_classe = {
        'at2':row.at2,
        'sro':row.sro,
        'inf':row.inf,
        'imo':row.imo,
        'ept':row.ept,
        'esp':row.esp,
        'fam':row.fam,
        'tvt':row.tvt,
        'cid':row.cid,
        'mul':row.mul,
        'pot':row.pot,
        'tav':row.tav,
        'mic':row.mic,
        'opi':row.opi,
        'poc':row.poc,
        'reg':row.reg,
        'eco':row.eco,
        'bro':row.bro,
        'cit':row.cit,
        'con':row.con,
        'int':row.int
    }
    return dic_classe[f'{classe}']
    

In [32]:
# conta quantas classes o termo ki aparece
def num_ki_em_categoria():

    dic_num_ki_em_categoria ={}
    for row in tqdm(df_relacao_class.itertuples(index=True, name='Pandas')):
        ci = 0
        for classe in classes:
            if get_row_class(row,classe)!= 0:
                ci += 1
        dic_num_ki_em_categoria[f'{row.Index}'] = ci
        
    return dic_num_ki_em_categoria

# Bag of Word

In [33]:
bag_of_word = {}
bag_of_word['a'] = df_a
bag_of_word['b'] = df_b
bag_of_word['c'] = df_c
bag_of_word['d'] = df_d
bag_of_word['freq_pal_class'] = df_relacao_class
bag_of_word['ci'] = num_ki_em_categoria()
bag_of_word['ni'] = dic_ni

9561it [00:00, 29980.73it/s]


## ICF

In [34]:
import math 

In [35]:
def calculeteICF():
    dic_num_ki_em_categoria = bag_of_word['ci']
    icfs = {}
    C = len(classes)
    for key in dic_num_ki_em_categoria:
        Ci = dic_num_ki_em_categoria[key]
        valor = math.log((1 + (C/Ci)))
        icfs[f'{key}'] = valor
    return icfs

In [36]:
icfs = calculeteICF() 
icfs

{'acusado': 2.4423470353692043,
 'trafico': 2.0794415416798357,
 'morto': 3.091042453358316,
 'maria': 1.3862943611198906,
 'ortiz': 3.091042453358316,
 'israel': 3.091042453358316,
 'pinheiro': 3.091042453358316,
 'lacerda': 3.091042453358316,
 'anos': 1.2039728043259361,
 'manha': 1.5040773967762742,
 'ontem': 1.2039728043259361,
 'bairro': 1.8325814637483102,
 'vitoria': 1.067840630001356,
 'ele': 1.1314021114911006,
 'atingido': 3.091042453358316,
 'tiros': 3.091042453358316,
 'pistola': 3.091042453358316,
 '765': 3.091042453358316,
 'atingiram': 3.091042453358316,
 'bracos': 2.4423470353692043,
 'costas': 3.091042453358316,
 'cabeca': 2.0794415416798357,
 'assassinato': 3.091042453358316,
 'aconteceu': 1.8325814637483102,
 'volta': 1.3862943611198906,
 'horas': 1.2039728043259361,
 'rua': 1.2878542883066382,
 'penha': 1.5040773967762742,
 'costa': 1.2878542883066382,
 'rocha': 1.8325814637483102,
 'acordo': 1.2878542883066382,
 'investigadores': 2.4423470353692043,
 'divisao': 2.4

In [37]:
df_icf = pd.DataFrame()
df_icf['Index'] = df_text_clean.index

for termo in tqdm(lista_palavras):
    
    list_icfs = []
    
    for i in df_text_clean.index:
        doc = df_text_clean.iloc[i].Text
        if termo in doc:
            list_icfs.append(icfs[termo])
        else:
            list_icfs.append(0)
    
    df_icf[f'{termo}'] = list_icfs
    
df_icf = df_icf.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████| 9561/9561 [02:32<00:00, 62.57it/s]


In [38]:
df_icf.head()

,acusado,trafico,morto,maria,ortiz,israel,pinheiro,lacerda,anos,manha,...,performance,leitura,envio,chamadas,telefonicas,oferecemos,sofisticados,inteligentes,violet,14/08/2006
Index,,,,,,,,,,,,,,,,,,,,,
0,2.442347,2.079442,3.091042,1.386294,3.091042,3.091042,3.091042,3.091042,1.203973,1.504077,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.203973,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.203973,1.504077,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.504077,...,3.091042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

## X²

In [39]:
qui_quadrado = pd.DataFrame()
qui_quadrado['Index'] = lista_palavras

for classe in tqdm(classes):
    lista_valores = []
    for termo in lista_palavras:
        a = bag_of_word['a'].loc[termo, classe]
        b = bag_of_word['b'].loc[termo, classe]
        c = bag_of_word['c'].loc[termo, classe]
        d = bag_of_word['d'].loc[termo, classe]
        
        numerador = (len(df_text_clean) * math.pow(((a * d) - (c * d)), 2))
        denominador = ((a + c) * (b + d) * (a + b) * (c + d))

        if (numerador == 0):
            value = 0
        else:
            value = numerador/denominador

        lista_valores.append(value)

    qui_quadrado[f'{classe}'] = lista_valores

qui_quadrado = qui_quadrado.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:06<00:00,  3.09it/s]


In [40]:
qui_quadrado.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,66.177211,7.875779,30.718625,0,166.863812,0,0,0,52.385474,0,...,0,0,52.385474,15.815481,52.385474,92.195370,30.718625,45.293795,0,85.905421
trafico,174.325867,20.564852,80.466303,0,450.396816,0,0,0,137.678383,0,...,0,0,71.265791,327.613958,137.678383,244.036035,80.466303,118.906485,0,293.182498
morto,204.968484,24.160090,94.561520,0,530.746521,0,0,0,161.844938,0,...,0,0,161.844938,386.346648,161.844938,287.059098,94.561520,139.763558,0,268.212648
maria,54.959853,10.608810,41.433464,0,198.906311,0,0,0,36.649047,0,...,0,0,13.999194,232.852245,36.649047,90.848988,41.433464,61.148684,0,149.718936
ortiz,1277.467327,149.993421,587.894756,0,3343.210808,0,0,0,1007.677735,0,...,0,0,1007.677735,3832.647217,1007.677735,1792.888847,587.894756,869.763222,0,2158.317102


---

## Relevance Frequency (RF)

In [41]:
def CalculateRF():
    df_rf = pd.DataFrame()
    df_rf['Index'] = lista_palavras
    for classe in classes:
        valores = []
        for termo in lista_palavras:
            a =  bag_of_word['a'].loc[termo,classe]
            b =  bag_of_word['b'].loc[termo,classe]
            value =  math.log(2 + (a / (max(1,b))))
            valores.append(value)
        df_rf[f'{classe}'] = valores
            
    df_rf =  df_rf.set_index('Index')
    return (df_rf)

In [42]:
df_rf = CalculateRF()
df_rf.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,1.897120,0.693147,0.693147,0.693147,0.693147,0.693147,0.723919
trafico,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.773190,1.504077,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
morto,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,1.945910,0.693147,0.693147,0.693147,0.693147,0.693147,0.788457
maria,0.733969,0.693147,0.693147,0.693147,0.733969,0.693147,0.693147,0.693147,0.733969,0.693147,...,0.693147,0.693147,0.965081,0.780159,0.733969,0.733969,0.693147,0.693147,0.693147,0.693147
ortiz,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147


---

## ICF-Based

In [43]:
def Calculate_ICF_BASED():
       
    icf_based = pd.DataFrame()
    icf_based['Index'] = lista_palavras
    
    C = len(classes)
    dic_num_ki_em_categoria = bag_of_word['ci']
    
    for classe in tqdm(classes):
        valores = []
        
        for termo in lista_palavras:
            a = bag_of_word['a'].loc[termo, classe]
            b = bag_of_word['b'].loc[termo, classe]
            Ci = dic_num_ki_em_categoria[termo]

            valor = math.log((2 + (a/max(1,b)) * (C/Ci)))
            valores.append(valor)
            
        icf_based[f'{classe}'] = valores
    
    icf_based = icf_based.set_index('Index')
    
    return icf_based

In [44]:
icf_based = Calculate_ICF_BASED()
icf_based.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:03<00:00,  6.68it/s]


,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,3.931826,0.693147,0.693147,0.693147,0.693147,0.693147,0.976915
trafico,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,1.152680,2.970414,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
morto,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,4.672829,0.693147,0.693147,0.693147,0.693147,0.693147,1.824549
maria,0.810930,0.693147,0.693147,0.693147,0.810930,0.693147,0.693147,0.693147,0.810930,0.693147,...,0.693147,0.693147,1.354546,0.934309,0.810930,0.810930,0.693147,0.693147,0.693147,0.693147
ortiz,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,3.135494,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147


---

## RF*IDF

In [45]:
def CalculateIDF():
    idfDict = {}
    N = len(df_text_clean)

    for termo, value in tqdm(bag_of_word['ni'].items()):
        idfDict[termo] = math.log10(N / (float(value)))
        
    return(idfDict)

In [46]:
idfDict = CalculateIDF()

100%|█████████████████████████████████████████████████████████████████████████| 9561/9561 [00:00<00:00, 1577442.39it/s]


In [47]:
def CalculateRF_IDF():
    df_rfIdf = pd.DataFrame()
    df_rfIdf['Index'] = df_rf.index
    
    for classe in classes:
        valores = []
        for index in df_rf.index:            
            valueRF =  df_rf.loc[index,classe]
            valueIdf =  idfDict[index]
            value =  valueRF * valueIdf
            valores.append(value)
        df_rfIdf[f'{classe}'] = valores
            
    df_rfIdf =  df_rfIdf.set_index('Index')
    return (df_rfIdf)

In [48]:
df_rfIdf = CalculateRF_IDF()
df_rfIdf.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
acusado,0.659457,0.659457,0.659457,0.659457,0.659457,0.659457,0.659457,0.659457,0.659457,0.659457,...,0.659457,0.659457,0.659457,1.804910,0.659457,0.659457,0.659457,0.659457,0.659457,0.688733
trafico,0.926561,0.926561,0.926561,0.926561,0.926561,0.926561,0.926561,0.926561,0.926561,0.926561,...,0.926561,0.926561,1.033558,2.010569,0.926561,0.926561,0.926561,0.926561,0.926561,0.926561
morto,0.972965,0.972965,0.972965,0.972965,0.972965,0.972965,0.972965,0.972965,0.972965,0.972965,...,0.972965,0.972965,0.972965,2.731459,0.972965,0.972965,0.972965,0.972965,0.972965,1.106752
maria,0.783806,0.740212,0.740212,0.740212,0.783806,0.740212,0.740212,0.740212,0.783806,0.740212,...,0.740212,0.740212,1.030610,0.833132,0.783806,0.783806,0.740212,0.740212,0.740212,0.740212
ortiz,1.512339,1.512339,1.512339,1.512339,1.512339,1.512339,1.512339,1.512339,1.512339,1.512339,...,1.512339,1.512339,1.512339,2.397000,1.512339,1.512339,1.512339,1.512339,1.512339,1.512339


---

## Machine Learning

### KNN

In [49]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve,classification_report,r2_score
from sklearn import model_selection as ms
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [50]:
le = LabelEncoder()

In [51]:
def toNumeric(nome_coluna, coluna):
    df_text_clean[nome_coluna] = le.fit_transform (coluna)

In [52]:
toNumeric('numeric_class',df_text_clean.Class)

In [53]:
# features
X = TF

In [54]:
#response
Y = df_text_clean['numeric_class'].copy()

In [55]:
x_train, x_test, y_train, y_test = ms.train_test_split(X,Y,test_size=0.33, random_state=5)

In [56]:
model_knn = KNeighborsClassifier(n_neighbors=1)

In [57]:
model_knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [58]:
pred = model_knn.predict_proba(x_test)

In [59]:
plot_roc_curve(np.array(y_test), pred[:,1])

NameError: name 'plot_roc_curve' is not defined

# Imports do EDA

In [ ]:
# print da curva ROC
def plot_roc_curve(y_true, y_score, figsize=(10,6)):
    fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=2)
    plt.figure(figsize=figsize)
    auc_value = roc_auc_score(y_true, y_score)
    plt.plot(fpr, tpr, color='orange', label='ROC curve (area = %0.2f)' % auc_value)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('Taxa de falso positivo')
    plt.ylabel('Taxa de verdadeiro positivo')
    plt.title('Curva ROC')
    plt.legend()
    lt.show()

In [ ]:
#precision    recall  f1-score   support
print(classification_report(y_test, pred, labels=[1, 2, 3]))

In [ ]:
# AUC
print('AUC: %0.2f' % roc_auc_score(y_test, pred))

## Número de amostras de cade classe


In [ ]:
import seaborn as sns

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(x = 'Class',data = textDataDrame)